In [1]:
import pandas as pd
import os, sys
source_path = os.path.join(os.getcwd(), os.pardir, 'preprocessing')
sys.path.append(source_path)

from preprocess import (
    show_df, 
    date_features, 
    preprocess, 
    ToTorch, 
    get_loader, 
    ts_split,
    cont_cat_split,
    dataloader_by_stock,
    get_train_data,
    TextTransform
)


C:\Users\gilbe\anaconda3\envs\tokyo\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
computer_name1 = 'gilbe'
computer_name2 = 'Gilberto-BE'

ROOT_PATH = f'c:/Users/{computer_name1}/Documents/TokyoData'

In [3]:
stock_list = pd.read_csv(f'{ROOT_PATH}/stock_list.csv')
print(stock_list.info())
print()
print(stock_list.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4417 entries, 0 to 4416
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   SecuritiesCode          4417 non-null   int64  
 1   EffectiveDate           4417 non-null   int64  
 2   Name                    4417 non-null   object 
 3   Section/Products        4417 non-null   object 
 4   NewMarketSegment        3772 non-null   object 
 5   33SectorCode            4417 non-null   object 
 6   33SectorName            4417 non-null   object 
 7   17SectorCode            4417 non-null   object 
 8   17SectorName            4417 non-null   object 
 9   NewIndexSeriesSizeCode  4417 non-null   object 
 10  NewIndexSeriesSize      4417 non-null   object 
 11  TradeDate               4121 non-null   float64
 12  Close                   4121 non-null   float64
 13  IssuedShares            4121 non-null   float64
 14  MarketCapitalization    4121 non-null   

In [4]:



'/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv'
'/train_files/trades.csv'

train_df = pd.read_csv(f'{ROOT_PATH}/train_files/stock_prices.csv')
train_df['Date'] = pd.to_datetime(train_df['Date']) 
train_df.set_index('Date', inplace=True)
# train_df = date_features(train_df)
print(train_df.head(2))

train_options = pd.read_csv(f'{ROOT_PATH}/train_files/options.csv', low_memory=False)
train_financials = pd.read_csv(f'{ROOT_PATH}/train_files/financials.csv', low_memory=False)
train_trades = pd.read_csv(f'{ROOT_PATH}/train_files/trades.csv', low_memory=False)


print(train_df.head(2))
print(train_df.info())
print('.' * 20)
print()



print('OPTIONS')
print(train_options.head(2))
print(train_options.info())
print('.' * 50)
print()


print('FINANCIALS')
print(train_financials.head(2))
print(train_financials.info())
print('.' * 50)
print()


print('TRADES')
print(train_trades.head(2))
print(train_trades.info())
print('.' * 50)
print()


                    RowId  SecuritiesCode    Open    High     Low   Close  \
Date                                                                        
2017-01-04  20170104_1301            1301  2734.0  2755.0  2730.0  2742.0   
2017-01-04  20170104_1332            1332   568.0   576.0   563.0   571.0   

             Volume  AdjustmentFactor  ExpectedDividend  SupervisionFlag  \
Date                                                                       
2017-01-04    31400               1.0               NaN            False   
2017-01-04  2798500               1.0               NaN            False   

              Target  
Date                  
2017-01-04  0.000730  
2017-01-04  0.012324  
                    RowId  SecuritiesCode    Open    High     Low   Close  \
Date                                                                        
2017-01-04  20170104_1301            1301  2734.0  2755.0  2730.0  2742.0   
2017-01-04  20170104_1332            1332   568.0   576.0   563

# Merging data

### Ideas
Train some stocks from some industries

In [5]:
stock_list[['SecuritiesCode', 'MarketCapitalization']]

,SecuritiesCode,MarketCapitalization
0,1301,3.365911e+10
1,1305,7.621831e+12
2,1306,1.641739e+13
3,1308,7.671945e+12
4,1309,3.216145e+09
...,...,...
4412,9994,2.654474e+10
4413,9995,1.083499e+10
4414,9996,1.361913e+10
4415,9997,6.894633e+10


In [6]:
merged_data = stock_list.merge(train_df, on='SecuritiesCode')

In [7]:
print(merged_data[merged_data['SecuritiesCode'] == 1301].shape)
display(merged_data[merged_data['SecuritiesCode'] == 1301])


(1202, 26)


,SecuritiesCode,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,...,RowId,Open,High,Low,Close_y,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,20170104_1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,20170105_1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,NaN,False,0.002920
2,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,20170106_1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,NaN,False,-0.001092
3,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,20170110_1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,NaN,False,-0.005100
4,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,20170111_1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,NaN,False,-0.003295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,20211129_1301,2975.0,2984.0,2951.0,2951.0,14300,1.0,NaN,False,0.003793
1198,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,20211130_1301,2953.0,2997.0,2900.0,2900.0,20500,1.0,NaN,False,0.007558
1199,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,20211201_1301,2909.0,2936.0,2909.0,2911.0,10600,1.0,NaN,False,0.016706
1200,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,20211202_1301,2949.0,2973.0,2933.0,2933.0,15200,1.0,NaN,False,-0.003689


In [8]:
stock_list['Section/Products'].unique()

array(['First Section (Domestic)', 'ETFs/ ETNs',
       'JASDAQ(Standard / Domestic)', 'JASDAQ(Growth/Domestic)',
       'Mothers (Domestic)', 'Second Section(Domestic)',
       'First Section (Foreign)',
       'REIT, Venture Funds, Country Funds and Infrastructure Funds',
       'JASDAQ(Standard / Foreign)', 'Mothers (Foreign)',
       'Second Section(Foreign)'], dtype=object)

In [9]:
stock_list['NewMarketSegment'].unique()

array(['Prime Market', nan, 'Standard Market', 'Growth Market',
       'Prime Market (Foreign Stock)', 'Standard Market (Foreign Stock)',
       'Growth Market (Foreign Stock)'], dtype=object)

In [10]:
stock_list['IssuedShares'].unique()

array([1.09282830e+07, 3.63463562e+09, 7.91771845e+09, ...,
       9.15264000e+06, 9.72444720e+07, 3.42469620e+07])

In [11]:
print(stock_list.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4417 entries, 0 to 4416
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   SecuritiesCode          4417 non-null   int64  
 1   EffectiveDate           4417 non-null   int64  
 2   Name                    4417 non-null   object 
 3   Section/Products        4417 non-null   object 
 4   NewMarketSegment        3772 non-null   object 
 5   33SectorCode            4417 non-null   object 
 6   33SectorName            4417 non-null   object 
 7   17SectorCode            4417 non-null   object 
 8   17SectorName            4417 non-null   object 
 9   NewIndexSeriesSizeCode  4417 non-null   object 
 10  NewIndexSeriesSize      4417 non-null   object 
 11  TradeDate               4121 non-null   float64
 12  Close                   4121 non-null   float64
 13  IssuedShares            4121 non-null   float64
 14  MarketCapitalization    4121 non-null   

In [12]:
print(stock_list.shape)

(4417, 16)


In [13]:
# !jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000
print(stock_list['NewMarketSegment'])
print()
token_list = list(stock_list['NewMarketSegment'])

print('token_list[:20]:')
print(token_list[:20])
txt_trans = TextTransform(token_list)
print(txt_trans.transform())



0          Prime Market
1                   NaN
2                   NaN
3                   NaN
4                   NaN
             ...       
4412    Standard Market
4413       Prime Market
4414    Standard Market
4415       Prime Market
4416                NaN
Name: NewMarketSegment, Length: 4417, dtype: object

token_list[:20]:
['Prime Market', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'Prime Market']
[2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 2, 2, 6, 6, 6, 6, 6, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 2, 4, 2, 2, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0, 0, 4, 6, 2, 2, 2, 4, 2, 4, 2, 2, 0, 2, 4, 2, 0, 4, 4, 4, 0, 2, 4, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 6, 6, 6, 6, 6, 6, 6, 6, 4, 2, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,